In [15]:
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import tokenize
import pandas as pd
import nltk
import numpy as np
from random import seed
from random import randint
import random
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras import layers


np.set_printoptions(threshold=10000)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

# sg_w2v_model = Word2Vec.load('embeddingword2vec_100.model')

In [16]:
def text_preprocess(text):
    text = text.lower()
    text = remove_stopwords(text)
    text = nltk.sent_tokenize(text)
    return text

# def data_preprocess(inpath):
#     df = pd.read_csv(inpath)
#     df['sentence'] = None
#     for i in df.iterrows():
#         df['sentence'][i[0]] = text_preprocess(i[1]['text'])
#     return df

# if __name__ == "__main__":
#     df = data_preprocess('gutenberg.csv')
#     print(type(df[0]['text']))

In [17]:
df = pd.read_csv('gutenberg.csv')
df['sentence'] = df['text'].copy()

for i in df.iterrows():
    df['sentence'][i[0]] = text_preprocess(i[1]['text'])
    
df['label_pretext'] = [[] for x in range(18)]

<ipython-input-17-19231c565353>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentence'][i[0]] = text_preprocess(i[1]['text'])


In [19]:
seed(1)
coverage = 0.1

agg_input = []
agg_label = []

for row in df.iterrows():
#     df['label_pretext'][row[0]] = label
    wo_df = df.loc[df.author != row[1]['author']]
    replace_pool = wo_df['sentence']
    replace_pool = [item for sublist in replace_pool.values.tolist() for item in sublist]
    
    text_sentences = row[1]['sentence']
    text_len = len(text_sentences)
    
    final_input = []
    input_arr = []
    
    for i in range(text_len):
        input_arr.append(text_sentences[i])
        if (i + 1) % 20 == 0:
            final_input.append(input_arr)
            input_arr = []
            
    label_arr = []
    
    for idx in range(len(final_input)):
        replace_idx = randint(0, 19)
        
        # create golden label
        label = np.zeros(20, dtype = int)
        label[replace_idx] = 1
        label_arr.append(label)
        
        # get random replacement sentence
        sent_idx = randint(0, len(replace_pool)-1)
        sent = replace_pool[sent_idx]

        # replace
        final_input[idx][replace_idx] = sent
        
    agg_input = agg_input + final_input
    agg_label = agg_label + label_arr

In [20]:
print(agg_input[0])

['[emma jane austen 1816] volume chapter emma woodhouse, handsome, clever, rich, comfortable home happy disposition, unite best blessings existence; lived nearly twenty-one years world little distress vex her.', "youngest daughters affectionate, indulgent father; had, consequence sister's marriage, mistress house early period.", 'mother died long ago indistinct remembrance caresses; place supplied excellent woman governess, fallen little short mother affection.', "sixteen years miss taylor mr. woodhouse's family, governess friend, fond daughters, particularly emma.", 'land gold!', "miss taylor ceased hold nominal office governess, mildness temper hardly allowed impose restraint; shadow authority long passed away, living friend friend mutually attached, emma liked; highly esteeming miss taylor's judgment, directed chiefly own.", "real evils, indeed, emma's situation power having way, disposition think little herself; disadvantages threatened alloy enjoyments.", 'danger, however, present

In [7]:
for agg_idx in range(len(agg_input)):
    for sub_idx in range(len(agg_input[agg_idx])):
        agg_input[agg_idx][sub_idx] = list(tokenize(agg_input[agg_idx][sub_idx], lowercase=True))

In [8]:
model_input = []
element_input = []
for subarr in agg_input:
    for sent in subarr:
        model_vector = (np.mean([sg_w2v_model.wv[token] for token in sent], axis=0)).tolist()

        if type(model_vector) is list:
            vector = [float(vector_element) for vector_element in model_vector]
            
        element_input.append(vector)
    model_input.append(element_input)
    element_input = []

NameError: name 'sg_w2v_model' is not defined

In [21]:
print(len(agg_input))

4703


In [23]:
len(agg_label)

4703

In [24]:
agg_input[0]

['[emma jane austen 1816] volume chapter emma woodhouse, handsome, clever, rich, comfortable home happy disposition, unite best blessings existence; lived nearly twenty-one years world little distress vex her.',
 "youngest daughters affectionate, indulgent father; had, consequence sister's marriage, mistress house early period.",
 'mother died long ago indistinct remembrance caresses; place supplied excellent woman governess, fallen little short mother affection.',
 "sixteen years miss taylor mr. woodhouse's family, governess friend, fond daughters, particularly emma.",
 'land gold!',
 "miss taylor ceased hold nominal office governess, mildness temper hardly allowed impose restraint; shadow authority long passed away, living friend friend mutually attached, emma liked; highly esteeming miss taylor's judgment, directed chiefly own.",
 "real evils, indeed, emma's situation power having way, disposition think little herself; disadvantages threatened alloy enjoyments.",
 'danger, however, 

In [25]:
df_save = pd.DataFrame({'text': agg_input, 'label': agg_label})

In [30]:
for i in range(len(df_save)):
    df_save['label'][i] = np.where(df_save['label'][i]==1)[0][0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [31]:
df_save.to_csv('expand_pretext.csv')

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(model_input, agg_label, test_size = 0.2, random_state = 42)

X_train = np.asarray(X_train).reshape(15072, 100)
Y_train = np.asarray(Y_train).reshape(15072, 5)

X_test = np.asarray(X_test).reshape(3769, 100)
Y_test = np.asarray(Y_test).reshape(3769, 5)

ValueError: Found input variables with inconsistent numbers of samples: [0, 4703]

In [311]:
X_train = np.asarray(X_train)

In [312]:
X_train.shape

(15072, 5, 100)

In [195]:
model = Sequential()
model.add(layers.Embedding(input_dim=len(sg_w2v_model.wv)+1,
                output_dim=100,
                  input_length=X_arr.shape[1]))
model.add(layers.SpatialDropout1D(0.2))
model.add(layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(5, activation='softmax'))
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          4176500   
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 100, 100)          0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_35 (Dense)             (None, 5)                 505       
_________________________________________________________________
dense_36 (Dense)             (None, 5)                 30        
Total params: 4,257,435
Trainable params: 4,257,435
Non-trainable params: 0
_________________________________________________________________


In [196]:
history = model.fit(X_arr, Y_arr,
                    epochs=10,
                    batch_size=10)

Epoch 1/10
1508/1508 [==============================] - 187s 123ms/step - loss: 1.6131 - accuracy: 0.2068
Epoch 2/10
1508/1508 [==============================] - 182s 121ms/step - loss: 1.6104 - accuracy: 0.1962
Epoch 3/10
1508/1508 [==============================] - 194s 129ms/step - loss: 1.6103 - accuracy: 0.2006
Epoch 4/10
1508/1508 [==============================] - 198s 131ms/step - loss: 1.6099 - accuracy: 0.2050
Epoch 5/10
1508/1508 [==============================] - 204s 135ms/step - loss: 1.6097 - accuracy: 0.2084
Epoch 6/10
1508/1508 [==============================] - 210s 139ms/step - loss: 1.6099 - accuracy: 0.2034
Epoch 7/10
1508/1508 [==============================] - 196s 130ms/step - loss: 1.6100 - accuracy: 0.2007
Epoch 8/10
1508/1508 [==============================] - 196s 130ms/step - loss: 1.6100 - accuracy: 0.1981
Epoch 9/10
1508/1508 [==============================] - 196s 130ms/step - loss: 1.6100 - accuracy: 0.1949
Epoch 10/10
1508/1508 [=======================

In [213]:
loss, accuracy = model.evaluate(X_test, Y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.1998


In [ ]:
accuracy = model.evaluate(X_test, Y_test)
prediction = model.predict(X_test)
argmax_prediction = []
argmax_actual = []
for i in range(len(prediction)):
    print(prediction[i])
    break
    argmax_prediction.append(np.argmax(prediction[i]))
    argmax_actual.append(np.argmax(Y_test[i]))

print(argmax_prediction)
print(argmax_actual)
# print(Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accuracy[0],accuracy[1]))

Jane Austen

In [263]:
seed(1)
coverage = 0.1

agg_input = []
agg_label = []

wo_df = df.loc[df.author != 'austen']
replace_pool = wo_df['sentence']
replace_pool = [item for sublist in replace_pool.values.tolist() for item in sublist]

austen_df = df[df.author == 'austen']

for row in austen_df.iterrows():
    text_sentences = row[1]['sentence']
    text_len = len(text_sentences)
    
    final_input = []
    input_arr = []
    
    for i in range(text_len):
        input_arr.append(text_sentences[i])
        
        # break by 5 sentences each
        if (i + 1) % 5 == 0:
            final_input.append(input_arr)
            input_arr = []
            
    label_arr = []
    
    for idx in range(len(final_input)):
        replace_idx = randint(0, 4)
        
        # create golden label
        label = np.zeros(5, dtype = int)
        label[replace_idx] = 1
        label_arr.append(label)
        
        # get random replacement sentence
        sent_idx = randint(0, len(replace_pool)-1)
        sent = replace_pool[sent_idx]
        
        # replace
        final_input[idx][replace_idx] = sent
    
        
    agg_input = agg_input + final_input
    agg_label = agg_label + label_arr

In [264]:
for agg_idx in range(len(agg_input)):
    for sub_idx in range(len(agg_input[agg_idx])):
        agg_input[agg_idx][sub_idx] = list(tokenize(agg_input[agg_idx][sub_idx], lowercase=True))

In [271]:
sg_w2v_model.wv['word']

array([ 0.08622476, -0.1144415 ,  0.12604907,  0.07611374,  0.1835219 ,
       -0.21370359,  0.46409512,  0.6325722 , -0.08867782, -0.63324827,
        0.37734088, -0.51708555,  0.15839037,  0.170145  , -0.17042297,
       -0.20000845,  0.26558313, -0.40252045,  0.13463554, -0.62030476,
       -0.2038364 ,  0.5265013 ,  0.30034912,  0.13111448, -0.3278476 ,
       -0.01337765, -0.29178175,  0.13141643, -0.15398963, -0.25098673,
        0.39202592, -0.01120253,  0.09209356, -0.35319647,  0.08569322,
        0.5614367 ,  0.06050458, -0.4363827 , -0.13195081, -0.5483975 ,
       -0.38859499, -0.22050627, -0.31863436,  0.1643234 ,  0.2102585 ,
       -0.09756722, -0.36572388,  0.06141844,  0.22395839,  0.15812865,
       -0.01831647, -0.23212026, -0.01463716, -0.05991371,  0.03973402,
       -0.05622132,  0.3304451 , -0.16415226, -0.17690657,  0.15431233,
        0.01757321, -0.28130022,  0.21162207, -0.08450586, -0.33811277,
        0.52944744,  0.15356852,  0.28750575, -0.25274092,  0.19

In [272]:
model_input = []
element_input = []
for subarr in agg_input:
    try:
        model_vector = (np.mean([sg_w2v_model.wv[token] for token in sent], axis=0)).tolist()
    except:
        print(sent)
        
        
    if type(model_vector) is list:
        vector = [float(vector_element) for vector_element in model_vector]
            
    element_input.append(vector)
    model_input.append(element_input)
    element_input = []

34:4 host heaven shall dissolved, heavens shall rolled scroll: host shall fall down, leaf falleth vine, falling fig fig tree.
34:4 host heaven shall dissolved, heavens shall rolled scroll: host shall fall down, leaf falleth vine, falling fig fig tree.
34:4 host heaven shall dissolved, heavens shall rolled scroll: host shall fall down, leaf falleth vine, falling fig fig tree.
34:4 host heaven shall dissolved, heavens shall rolled scroll: host shall fall down, leaf falleth vine, falling fig fig tree.
34:4 host heaven shall dissolved, heavens shall rolled scroll: host shall fall down, leaf falleth vine, falling fig fig tree.
34:4 host heaven shall dissolved, heavens shall rolled scroll: host shall fall down, leaf falleth vine, falling fig fig tree.
34:4 host heaven shall dissolved, heavens shall rolled scroll: host shall fall down, leaf falleth vine, falling fig fig tree.
34:4 host heaven shall dissolved, heavens shall rolled scroll: host shall fall down, leaf falleth vine, falling fig fi